In [1]:
import numpy as np
import pandas as pd

import sklearn.linear_model as sk
import plotly.express as px
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier


from sklearn.decomposition import PCA


pd.set_option('max_columns', None)


#reading data and removing Nan values
data = pd.read_csv('FinancialDataSet.csv', usecols=np.r_[0:66])
#converting object to float
for col in data.columns: 
    data[col] = pd.to_numeric(data[col], errors='coerce')
 


data = data.fillna("0")


data


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\studies\\first sem docs\\BIGDATA\\bankruptcy_prediction\\FinancialDataSet.csv'

In [23]:
def preprocess_inputs(df):
    df = df.copy()
    y = df['class']
    X = df.drop(['class'], axis = 1)
#splitting the data
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, shuffle=True, random_state=1)
#scaling the data
    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train = pd.DataFrame(scaler.transform(X_train), index=X_train.index, columns=X_train.columns) #creating dataframe & combining x_traindata
    X_test = pd.DataFrame(scaler.transform(X_test), index=X_test.index, columns=X_test.columns) #creating dataframe & combining x_test data
    return X_train, X_test, y_train, y_test
   


In [24]:
X_train, X_test, y_train, y_test = preprocess_inputs(data)

In [52]:
original_models = {
    "Logistic Regression": LogisticRegression(),
    "RandomForestClassifier" :  RandomForestClassifier(),
    "Gradient Boosting": GradientBoostingClassifier()
    
}

for name, model in original_models.items():
    model.fit(X_train, y_train)
    print(name + " trained.")

C:\Users\bhagy\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression

Logistic Regression trained.
RandomForestClassifier trained.
Gradient Boosting trained.


In [54]:
original_results = []

for name, model in original_models.items():
    result = model.score(X_test, y_test)
    original_results.append(result)
    print(name + ": {:.2f}%".format(result * 100))

Logistic Regression: 95.14%
RandomForestClassifier: 97.05%
Gradient Boosting: 99.94%


In [55]:
y_test.value_counts() / len(y_test)

0     0.952385
1     0.047462
11    0.000154
Name: class, dtype: float64

In [75]:
n_components = 35

pca = PCA(n_components=n_components)
pca.fit(X_train)

X_train_reduced = pd.DataFrame(pca.transform(X_train), index=X_train.index, columns=["PC" + str(i) for i in range(1, n_components + 1)])
X_test_reduced = pd.DataFrame(pca.transform(X_test), index=X_test.index, columns=["PC" + str(i) for i in range(1, n_components + 1)])

In [76]:
X_train_reduced

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,PC11,PC12,PC13,PC14,PC15,PC16,PC17,PC18,PC19,PC20,PC21,PC22,PC23,PC24,PC25,PC26,PC27,PC28,PC29,PC30,PC31,PC32,PC33,PC34,PC35
39337,-0.018207,-0.091816,-0.043341,0.022086,-0.044058,-0.000357,0.042621,-0.031325,0.015961,0.008983,0.005934,-0.368680,0.096706,0.119721,0.782559,-0.592727,0.029059,-0.176308,0.846919,0.241247,-0.026397,0.184041,-0.309659,-0.063043,0.021112,-0.050029,-0.037522,0.004109,-0.008812,-0.004033,-0.003679,-0.004416,0.001717,-0.009222,-0.004598
32733,-0.006043,-0.006677,-0.004025,-0.101030,0.003125,0.026960,0.046211,-0.037370,0.091142,0.029256,-0.009276,-0.844516,0.046562,0.063104,0.444674,-0.416484,-0.009398,-0.081558,0.539132,0.345869,0.006707,0.139160,-0.211719,0.631417,0.015894,-0.136361,-0.005624,-0.000303,0.006517,0.017899,0.034604,-0.003083,-0.003383,-0.020467,-0.013847
35630,-0.023577,-0.254631,0.003733,0.224228,0.025463,0.013601,0.052387,-0.020120,-0.005670,-0.021638,0.000522,0.170128,0.052526,0.025809,0.567441,-0.480920,0.032738,-0.154169,0.683449,0.051981,0.029812,0.131271,-0.264927,-0.526987,0.200743,0.257548,-0.021835,0.007567,0.070764,-0.035419,0.127159,0.005126,0.024709,0.010061,0.017138
32427,-0.015426,-0.004466,-0.058430,-0.083580,-0.091469,0.009724,0.035080,-0.029485,0.043164,0.007909,-0.004000,-0.350565,0.049441,0.064746,0.479180,-0.367811,0.015818,-0.116359,0.506987,0.166061,-0.005591,0.119081,-0.185134,0.034936,-0.037305,-0.104775,-0.028781,-0.001358,0.016829,-0.004267,-0.049061,-0.005440,-0.022623,-0.007731,0.018305
21921,-0.022380,-0.150626,-0.071662,-0.082820,-0.098052,0.005512,-0.003085,-0.003262,0.003377,-0.021986,-0.012498,0.509784,0.003131,0.098337,-0.044446,-0.053117,0.004519,-0.029356,0.038576,-0.185862,-0.109099,-0.017077,0.034681,-0.363205,0.140289,0.208859,-0.005559,-0.000342,0.017545,-0.018679,0.071571,-0.004649,0.018105,0.010242,0.000331
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7813,-0.018176,-0.119934,-0.073907,-0.022652,-0.058623,-0.024219,-0.032955,0.004253,-0.038382,-0.029574,-0.022004,0.223008,-0.081750,-0.152589,-0.686362,0.409931,-0.019491,0.097038,-0.684267,-0.223331,0.053888,-0.169792,0.235405,-0.162377,0.032445,0.075852,0.011840,-0.011481,-0.029223,-0.007757,0.039409,0.005289,0.009776,0.003295,0.001338
32511,-0.014696,-0.015155,-0.060266,-0.108249,-0.187219,-0.002114,-0.066185,-0.034095,0.025006,-0.032777,-0.001374,0.655120,0.041967,0.082947,0.455010,-0.408804,0.026984,-0.126631,0.584225,-0.139419,-0.023357,0.110257,-0.153473,-0.480559,0.175355,0.284344,-0.015565,0.008100,-0.012301,-0.021572,-0.034457,0.001768,0.010200,0.010559,0.003751
5192,-0.013065,0.072967,-0.050757,0.074531,-0.049593,-0.004070,-0.017805,-0.007037,0.008099,-0.043889,-0.033323,0.359815,-0.142682,-0.141411,-0.861890,0.492450,-0.029518,0.124792,-0.809595,-0.254295,0.039917,-0.160544,0.259495,-0.282258,0.159232,0.231631,0.018067,-0.011459,0.008873,-0.012937,-0.042811,0.006621,0.005334,0.008089,-0.030521
12172,-0.013805,-0.040027,-0.059342,0.034084,-0.095904,-0.037198,-0.081596,-0.010236,-0.038844,-0.046189,-0.017524,0.521747,-0.092814,-0.086666,-0.505362,0.270265,-0.008446,0.048843,-0.452804,-0.277668,0.007442,-0.098138,0.157944,-0.412348,0.132631,0.237533,0.007614,-0.004890,-0.026129,-0.012780,-0.025739,0.007299,-0.014866,0.015166,-0.004720


In [77]:
X_train_reduced.describe()

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,PC11,PC12,PC13,PC14,PC15,PC16,PC17,PC18,PC19,PC20,PC21,PC22,PC23,PC24,PC25,PC26,PC27,PC28,PC29,PC30,PC31,PC32,PC33,PC34,PC35
count,3.038000e+04,3.038000e+04,3.038000e+04,3.038000e+04,3.038000e+04,3.038000e+04,3.038000e+04,3.038000e+04,3.038000e+04,3.038000e+04,3.038000e+04,3.038000e+04,3.038000e+04,3.038000e+04,3.038000e+04,3.038000e+04,3.038000e+04,3.038000e+04,3.038000e+04,3.038000e+04,30380.000000,3.038000e+04,3.038000e+04,3.038000e+04,3.038000e+04,3.038000e+04,3.038000e+04,3.038000e+04,3.038000e+04,3.038000e+04,3.038000e+04,3.038000e+04,3.038000e+04,3.038000e+04,3.038000e+04
mean,4.677701e-18,-1.871080e-18,9.355401e-19,9.355401e-18,7.016551e-19,-2.338850e-18,-1.871080e-18,2.338850e-19,2.338850e-18,-1.637195e-18,-2.338850e-19,-7.952091e-18,2.338850e-18,-7.016551e-19,-6.548781e-18,-9.355401e-19,7.016551e-19,-9.355401e-18,1.777526e-17,9.589286e-18,0.000000,6.548781e-18,-7.016551e-18,-1.379922e-17,3.274390e-18,3.742161e-18,1.169425e-18,9.355401e-19,-8.185976e-19,-1.637195e-18,-1.871080e-18,1.520253e-18,-2.455793e-18,-3.508276e-19,-5.847126e-19
std,3.334941e+00,3.298840e+00,2.216670e+00,2.148089e+00,1.802736e+00,1.648744e+00,1.540915e+00,1.493108e+00,1.386848e+00,1.299108e+00,1.233987e+00,1.098409e+00,1.027376e+00,1.021177e+00,1.009374e+00,1.001917e+00,1.000064e+00,9.996581e-01,9.963366e-01,9.867328e-01,0.979587,9.707967e-01,9.667075e-01,9.059941e-01,8.876284e-01,8.669272e-01,7.836200e-01,6.804673e-01,6.315545e-01,6.202831e-01,5.662040e-01,5.595335e-01,4.147146e-01,3.538724e-01,2.815011e-01
min,-5.425120e+01,-1.769364e+02,-2.526207e+01,-9.184092e+01,-2.370756e+01,-9.753875e+01,-8.435760e+01,-5.908563e+01,-8.188391e+01,-2.228986e+00,-2.642795e+01,-1.391913e+01,-3.160745e+01,-6.012955e+01,-3.269279e+01,-2.457679e+01,-2.071078e+01,-4.903698e+01,-4.878407e+01,-2.987395e+01,-63.985748,-6.939957e+01,-4.551679e+01,-2.252404e+01,-6.128162e+01,-5.749838e+01,-4.805139e+01,-1.706337e+01,-4.125010e+01,-1.469148e+01,-3.883726e+01,-3.680663e+01,-3.550694e+01,-5.485385e+00,-1.859767e+01
25%,-1.670564e-02,-1.036426e-01,-6.263856e-02,-5.654191e-02,-1.108568e-01,-2.609797e-02,-5.575304e-02,-3.125645e-02,-3.416832e-02,-3.197118e-02,-2.252192e-02,-4.868247e-01,-8.143005e-02,-1.014860e-01,-5.050545e-01,-3.740067e-01,-2.575278e-02,-1.193592e-01,-5.463885e-01,-1.735993e-01,-0.019737,-1.224642e-01,-1.911097e-01,-3.485367e-01,-1.370283e-01,-2.236025e-01,-2.717968e-02,-1.375735e-02,-1.723946e-02,-1.755595e-02,-1.577510e-02,-3.568936e-03,-2.506179e-02,-1.517617e-02,-8.621185e-03
50%,-1.305871e-02,-5.914673e-02,-5.207079e-02,-1.102797e-02,-5.826783e-02,-1.275411e-02,-1.112464e-02,-2.099583e-02,-1.052535e-02,-1.206288e-02,-1.118930e-02,-5.799707e-02,-1.362257e-02,-1.367051e-02,-4.735020e-03,-1.684481e-02,-3.760690e-03,-1.889332e-02,-1.251269e-02,-4.316616e-03,-0.000651,-2.124446e-04,-1.046267e-02,-3.325429e-02,2.576643e-03,2.579669e-03,-1.169803e-02,-5.538303e-03,4.818038e-04,-6.450773e-03,2.372827e-03,4.795922e-04,-4.778332e-03,-2.355490e-03,-3.078354e-04
75%,-9.320174e-03,1.362774e-03,-3.806431e-02,3.793784e-02,-9.362588e-04,4.179651e-03,4.019635e-02,-1.055744e-02,1.894798e-02,8.438778e-03,4.244033e-04,3.631733e-01,5.273083e-02,7.534419e-02,4.933845e-01,3.352918e-01,1.986603e-02,7.859780e-02,5.283291e-01,1.626946e-01,0.017417,1.221295e-01,1.710520e-01,3.144925e-01,1.350938e-01,2.145251e-01,4.471327e-03,3.469702e-03,2.106782e-02,4.312265e-03,2.019711e-02,4.383797e-03,1.570597e-02,9.545325e-03,6.657200e-03
max,5.697776e+02,4.293517e+02,2.696642e+02,1.567306e+02,1.653531e+02,1.563989e+02,1.372117e+02,1.697716e+02,9.444763e+01,2.072238e+02,1.839796e+02,5.670828e+01,1.236411e+02,1.155124e+02,9.607916e+01,7.863389e+01,1.611561e+02,7.773624e+01,6.778283e+01,6.972168e+01,106.989457,1.099025e+02,1.339622e+02,5.829705e+01,7.603761e+01,7.068179e+01,8.409973e+01,1.059589e+02,4.358034e+01,6.853062e+01,3.946139e+01,8.878602e+01,3.808033e+01,4.332773e+01,2.927866e+01


In [78]:
pca.explained_variance_ratio_

array([0.17109944, 0.16741514, 0.07559177, 0.0709867 , 0.04999618,
       0.04181951, 0.03652834, 0.0342969 , 0.02958897, 0.02596346,
       0.02342575, 0.01856098, 0.01623794, 0.01604257, 0.01567389,
       0.01544315, 0.01538607, 0.01537359, 0.0152716 , 0.01497861,
       0.01476246, 0.01449869, 0.01437681, 0.01262767, 0.0121209 ,
       0.01156212, 0.00944677, 0.00712339, 0.00613612, 0.00591905,
       0.00493195, 0.00481642, 0.00264589, 0.00192649, 0.00121908])

In [79]:
fig = px.bar(
    x=["PC" + str(i) for i in range(1, n_components + 1)],
    y=pca.explained_variance_ratio_,
    labels={'x': "Principal Component", 'y': "Variance Ratio"},
    color=pca.explained_variance_ratio_,
    color_continuous_scale=[(0, 'lightblue'), (1, 'darkblue')],
    title="Proportion of Variance in Principal Components"
)

fig.show()

In [80]:
reduced_models = {
    " Logistic Regression": LogisticRegression(),
    "RandomForestClassifier" : RandomForestClassifier(),
    "Gradient Boosting": GradientBoostingClassifier()
}
for name, model in reduced_models.items():
    model.fit(X_train_reduced, y_train)
    print(name + " trained.")

C:\Users\bhagy\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression

 Logistic Regression trained.
RandomForestClassifier trained.
Gradient Boosting trained.


In [82]:
reduced_results = []

for name, model in reduced_models.items():
    result = model.score(X_test_reduced, y_test)
    reduced_results.append(result)
    print(name + ": {:.2f}%".format(result * 100))

 Logistic Regression: 95.22%
RandomForestClassifier: 96.30%
Gradient Boosting: 95.98%


In [83]:
#change in performance After PCA
fig = px.bar(
    x=np.subtract(reduced_results, original_results),
    y=original_models.keys(),
    #orientation='h',
    labels={'x': "Change in Performance", 'y': "Model"},
    color=np.subtract(reduced_results, original_results),
    color_continuous_scale=[(0, 'red'), (1, 'blue')],
    title="Change in Model Performance After Dimensionality Reduction"
)

fig.show()
